<a href="https://colab.research.google.com/github/sakthiprasanth16/nifty50-stocks-analytics/blob/main/Preprocess_for_tableau.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# -----------------------------
# 1. LOAD DATA
# -----------------------------
stocks = pd.read_csv("all_stocks.csv")
sector = pd.read_csv("updated_sector.csv")

stocks['date'] = pd.to_datetime(stocks['date'])
stocks = stocks[stocks['date'] >= "2023-11-22"]

# -----------------------------
# 2. DAILY RETURN (pct_change)
# -----------------------------
stocks = stocks.sort_values(['ticker', 'date'])
stocks['daily_return'] = stocks.groupby('ticker')['close'].pct_change()

# -----------------------------
# 3. CUMULATIVE RETURN
# -----------------------------
# Fixed: Changed .apply() to .transform() to ensure index alignment
stocks['cumulative_return'] = (
    stocks.groupby('ticker')['daily_return']
    .transform(lambda x: (1 + x.fillna(0)).cumprod() - 1)
)

# -----------------------------
# 4. AGGREGATED METRICS (PER TICKER)
# -----------------------------
metrics = (
    stocks.groupby('ticker')
    .agg(
        yearly_return = ('daily_return', lambda x: (1 + x.dropna()).prod() - 1),
        volatility = ('daily_return', 'std'),          # SAME AS Streamlit
        avg_price = ('close', 'mean'),
        avg_volume = ('volume', 'mean'),
        final_cumulative_return = ('cumulative_return', 'last')
    )
    .reset_index()
)

# -----------------------------
# 5. MONTHLY RETURN
# -----------------------------
stocks['MonthYear'] = stocks['date'].dt.to_period('M')

monthly_returns = (
    stocks.groupby(['ticker', 'MonthYear'])['close']
    .agg(lambda x: (x.iloc[-1] - x.iloc[0]) / x.iloc[0])
    .reset_index(name='monthly_return')
)

# -----------------------------
# 6. MERGE EVERYTHING
# -----------------------------
final_df = (
    stocks
    .merge(metrics, on='ticker', how='left')
    .merge(sector, left_on='ticker', right_on='symbol', how='left')
)

# -----------------------------
# 7. SAVE FOR TABLEAU
# -----------------------------
final_df.to_csv("tableau_ready_stock_metrics.csv", index=False)

print("✅ Exported: tableau_ready_stock_metrics.csv")
print("Columns:")
print(final_df.columns.tolist())

✅ Exported: tableau_ready_stock_metrics.csv
Columns:
['date', 'open', 'close', 'high', 'low', 'volume', 'ticker', 'daily_return', 'cumulative_return', 'MonthYear', 'yearly_return', 'volatility', 'avg_price', 'avg_volume', 'final_cumulative_return', 'company', 'sector', 'symbol']


In [ ]:
display(final_df.head())

,date,open,close,high,low,volume,ticker,daily_return,cumulative_return,MonthYear,yearly_return,volatility,avg_price,avg_volume,final_cumulative_return,company,sector,symbol
0,2023-11-22,2205.15,2172.65,2225.00,2153.5,1247205,ADANIENT,NaN,0.000000,2023-11,0.025476,0.029948,3042.43253,2.495226e+06,0.025476,ADANI ENTERPRISES,MISCELLANEOUS,ADANIENT
1,2023-11-23,2190.00,2175.25,2190.00,2160.6,729114,ADANIENT,0.001197,0.001197,2023-11,0.025476,0.029948,3042.43253,2.495226e+06,0.025476,ADANI ENTERPRISES,MISCELLANEOUS,ADANIENT
2,2023-11-24,2179.00,2225.45,2251.95,2160.9,1946259,ADANIENT,0.023078,0.024302,2023-11,0.025476,0.029948,3042.43253,2.495226e+06,0.025476,ADANI ENTERPRISES,MISCELLANEOUS,ADANIENT
3,2023-11-28,2301.00,2423.50,2512.40,2275.0,11972275,ADANIENT,0.088993,0.115458,2023-11,0.025476,0.029948,3042.43253,2.495226e+06,0.025476,ADANI ENTERPRISES,MISCELLANEOUS,ADANIENT
4,2023-11-29,2468.70,2396.60,2477.65,2390.0,4561574,ADANIENT,-0.011100,0.103077,2023-11,0.025476,0.029948,3042.43253,2.495226e+06,0.025476,ADANI ENTERPRISES,MISCELLANEOUS,ADANIENT


In [ ]:
import pandas as pd

# --------------------------------------------------
# STEP 1: LOAD YOUR FINAL LOCAL FILE
# --------------------------------------------------
FILE_PATH = "/content/tableau_ready_stock_metrics.csv"

df = pd.read_csv(FILE_PATH)

print("✅ File loaded successfully")
print(df.head())

# --------------------------------------------------
# STEP 2: DATA CLEANING & TYPE FIXES
# --------------------------------------------------
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['ticker', 'date'])

required_cols = {'date', 'ticker', 'close'}
if not required_cols.issubset(df.columns):
    raise ValueError(f"Missing required columns: {required_cols}")

print("✅ Data types and sorting done")

# --------------------------------------------------
# STEP 3: CREATE PRICE PIVOT TABLE
# --------------------------------------------------
price_pivot = df.pivot_table(
    index='date',
    columns='ticker',
    values='close'
)

print("✅ Price pivot table created")
print(price_pivot.head())

# --------------------------------------------------
# STEP 4: RETURNS-BASED CORRELATION (RECOMMENDED)
# --------------------------------------------------
returns = price_pivot.pct_change()
corr = returns.corr().round(2)

print("✅ Correlation matrix computed")

# --------------------------------------------------
# STEP 5: CONVERT TO LONG FORMAT (FIXED)
# --------------------------------------------------

# Rename index and columns to avoid duplicate names
corr.index.name = "RowTicker"
corr.columns.name = "ColTicker"

corr_long = corr.stack().reset_index(name="Correlation")

print("✅ Correlation converted to long format")
print(corr_long.head())

# --------------------------------------------------
# STEP 6: EXPORT TABLEAU-READY CSV
# --------------------------------------------------
OUTPUT_FILE = "stock_correlation_long_tableau.csv"

corr_long.to_csv(OUTPUT_FILE, index=False)

print(f"✅ Exported successfully: {OUTPUT_FILE}")
print("📌 Load this file directly into Tableau")


✅ File loaded successfully
         date     open    close     high     low    volume    ticker  \
0  2023-11-22  2205.15  2172.65  2225.00  2153.5   1247205  ADANIENT   
1  2023-11-23  2190.00  2175.25  2190.00  2160.6    729114  ADANIENT   
2  2023-11-24  2179.00  2225.45  2251.95  2160.9   1946259  ADANIENT   
3  2023-11-28  2301.00  2423.50  2512.40  2275.0  11972275  ADANIENT   
4  2023-11-29  2468.70  2396.60  2477.65  2390.0   4561574  ADANIENT   

   daily_return  cumulative_return MonthYear  yearly_return  volatility  \
0           NaN           0.000000   2023-11       0.025476    0.029948   
1      0.001197           0.001197   2023-11       0.025476    0.029948   
2      0.023078           0.024302   2023-11       0.025476    0.029948   
3      0.088993           0.115458   2023-11       0.025476    0.029948   
4     -0.011100           0.103077   2023-11       0.025476    0.029948   

    avg_price    avg_volume  final_cumulative_return            company  \
0  3042.43253 